## law_1.docx, law_2.docx 문서 두개
- pinecone store 저장 (하나의 인덱스에 저장)
    - index name 임의
- RetrievalQA 구현
    - prompt: rlm/rag-prompt
    - 질문: 전세사기


In [1]:

import os
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader1 = Docx2txtLoader('law_1.docx')
loader2 = Docx2txtLoader('law_2.docx')

data1 = loader1.load()
data2 = loader2.load()

all_docs = data1 + data2

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = text_splitter.split_documents(all_docs)

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

llm = ChatOpenAI()



c:\Users\71\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name='laws',
# )

database = PineconeVectorStore(
    index=pc.Index('laws'),
    embedding=embedding,
)

In [3]:
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {'context': database.as_retriever() | format_docs,
     'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke('전세사기 피해주택 임대인의 국세 계산은 어떻게 하나요? 식이 따로 있나요?')


'전세사기피해주택 임대인의 국세는 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액과 전세사기피해주택의 가격, 그리고 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액에 따라 산정됩니다. 국세의 안분 방법에 대한 세부적인 사항은 대통령령에 의해 정해집니다. 국세의 우선 징수에 대한 특례나 국세의 안분 방법에 대한 세부 내용은 대통령령에 의해 규정됩니다.'